In [0]:
# Librerías

from pyspark.sql.functions import (
    col, from_json, explode, arrays_zip, to_timestamp,  
    date_format, hour, expr, current_timestamp, lit
)
from pyspark.sql.types import StructType, StructField, StringType, ArrayType, DoubleType
import logging
import pandas as pd

In [0]:
# Logger para avisar

logging.basicConfig(level=logging.INFO, format='%(asctime)s | %(levelname)s | %(message)s', force=True)
logger = logging.getLogger("ETL_SILVER")

# Rutas de las tablas Bronze

TABLE_BRONZE_NASA = "fire_risk_project.01_bronze.bronze_nasa_firms"

# Ruta tabla Silver

TABLE_SILVER_NASA = "fire_risk_project.02_silver.silver_nasa_firms"

In [0]:
# Transformación de datos

df_nasa_raw = spark.read.table(TABLE_BRONZE_NASA)

df_nasa_silver = (df_nasa_raw
    # A. Tipos de datos
    # Convertimos latitud/longitud de texto a números reales para poder hacer cálculos geográficos
    .withColumn("latitude", col("latitude").cast("double"))    # Transforma a flotantes con 8 decimales
    .withColumn("longitude", col("longitude").cast("double"))  # Transforma a flotantes con 8 decimales
    
    # B. Normalización de hora
    # La NASA envía la hora como entero, usamos 'lpad' para rellenar con ceros a la izquierda hasta tener 4 dígitos
    .withColumn("time_str", expr("lpad(cast(acq_time as string), 4, '0')"))
    
    # C. Timestamp (acq_dat + time_str). Fecha + " " + Hora(2 chars) + ":" + Minutos(2 chars) + ":00"
    .withColumn("timestamp_incendio", to_timestamp(
        expr("concat(cast(acq_date as string), ' ', substr(time_str, 1, 2), ':', substr(time_str, 3, 2), ':00')"),
        "yyyy-MM-dd HH:mm:ss"
    ))
    
    # D. Drop Duplicates. Eliminamos duplicados por fecha/hora/lat/lon
    .dropDuplicates(['acq_date', 'acq_time', 'latitude', 'longitude'])
    
    # E. Claves para el cruce (Join) posterior
    .withColumn("fecha_join", col("acq_date"))
    .withColumn("hora_join", hour(col("timestamp_incendio"))) # Extraemos la hora (0-23)
)

# Guardado de tabla
df_nasa_silver.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(TABLE_SILVER_NASA)

logger.info(f"Tabla NASA guardada en {TABLE_SILVER_NASA}")

In [0]:
# Verificación 

df_nasa_check = spark.read.table(TABLE_SILVER_NASA)
display(df_nasa_check.limit(5))

In [0]:
%sql

SELECT count(*) FROM fire_risk_project.`02_silver`.silver_nasa_firms;

In [0]:
# Finalizado el proceso tranformación de datos para datos NASA FIRMS